In [ ]:
#pip install konlpy

In [ ]:
# GPU 연결 상태 확인
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if 'failed' in gpu_info:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [34]:
import torch
import os
import numpy as np
import pandas as pd
import random as rd
import torch.nn as nn
import torch.optim as optim
from konlpy.tag import Okt
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from google.colab import drive
from collections import Counter


In [4]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')

Mounted at /content/drive


In [5]:
raw_data = pd.read_csv('AI_chatbot(4th_prj)/data/preprocessed_qna_hs_v1.csv')
raw_data.head()

,questions,answers
0,숙이네닭발의 주소는 어떻게 되나요?,숙이네닭발의 주소는 서울 용산구 우사단로10길 47입니다
1,숙이네닭발의 영업시간은 몇시부터 몇시까지로 기재되어 있나요?,숙이네닭발의 영업시간은 18:00 - 23:40입니다
2,숙이네닭발의 연락처가 어떻게 되나요?,숙이네닭발의 연락처는 02-798-0838입니다
3,숙이네닭발에 인접한 시설을 알 수 있나요?,"숙이네닭발에 인접한 시설에는 이태원역 3번 출구, 보광초등학교가 있습니다"
4,숙이네닭발의 영업시간은 어떻게 되나요?,숙이네닭발의 영업시간은 18:00 - 23:40입니다


In [ ]:
# raw_data.info()

In [ ]:
# raw_data['questions'].nunique()

In [6]:
duplicates = raw_data[raw_data['questions'].duplicated(keep=False)]
duplicates_sorted = duplicates.sort_values(by='questions').reset_index(drop=True)
duplicates_sorted.head()
# len(duplicates_sorted)

,questions,answers
0,국보순대곱창전골의 휴무일은 어떻게 되나요?,국보순대곱창전골의 영업시간은 알수없음입니다
1,국보순대곱창전골의 휴무일은 어떻게 되나요?,국보순대곱창전골의 휴무일은 알수없음입니다
2,금은돈의 주요 메뉴는 무엇이 있나요?,금은돈의 주차시설이 없습니다
3,금은돈의 주요 메뉴는 무엇이 있나요?,"금은돈의 메뉴에는 금돈 삼겹살, 가브리살이 있습니다"
4,나따오비까 대치점의 영업시간은 어떻게 되나요?,"나따오비까 대치점의 메뉴에는 에그클래식, 시그니처 라떼이 있습니다"


In [7]:
df = pd.DataFrame(raw_data)

main_keywords = ['주소', '영업시간', '연락처', '전화번호', '시설', '휴무일', '주차', '메뉴']

okt = Okt()

def preprocess_text(text):
  tokens = okt.pos(text)
  return ' '.join(word for word, pos in tokens if pos in 'Noun')

def extract_keywords(text, keywords):
  words = text.split()
  found_keywords = set(word for word in words if word in keywords)
  return found_keywords

def keyword_check(question, answer, keywords): # question 문장의 키워드가 answer 문장에 포함 안됐는지 확인
    q_keywords = extract_keywords(preprocess_text(question), keywords)
    a_keywords = extract_keywords(preprocess_text(answer), keywords)
    return q_keywords.issubset(a_keywords)     # 질문 키워드가 모두 답변에 포함되어 있는지 확인

df['check'] = df.apply(lambda row: keyword_check(row['questions'], row['answers'], main_keywords), axis=1)

In [ ]:
# f_df = df[df['check'] == False]
# f_df = f_df.drop(columns=['check'])
# f_df

In [8]:
df = df[df['check']]
df = df.drop(columns=['check'])
# df

In [9]:
df.to_csv('preprocessed_qna_hs_v1_edit.csv', index=False)

In [11]:
new_data = pd.read_csv('preprocessed_qna_hs_v1_edit.csv')
df = pd.DataFrame(new_data)

어휘 사전 생성

In [12]:
# 음절 단위 토큰화
def tokenize(text):
    return list(text)

def build_vocab(texts):
  vocab = Counter()
  for text in texts:
      vocab.update(text)
  return vocab

vocab = build_vocab(df['questions'].tolist() + df['answers'].tolist())
vocab_size = len(vocab) + 1

In [13]:
word2idx = {word: idx for idx, (word, _) in enumerate(vocab.items(), start=1)}  # 인덱스 1부터 시작
idx2word = {idx: word for word, idx in word2idx.items()}

In [14]:
def text_to_indices(text, word2idx):
    return [word2idx[word] for word in text if word in word2idx]

# 질문과 답변을 인덱스로 변환
questions_indices = [text_to_indices(tokenize(text), word2idx) for text in df['questions'].tolist()]
answers_indices = [text_to_indices(tokenize(text), word2idx) for text in df['answers'].tolist()]

# 텐서로 변환 및 패딩
questions_tensors = [torch.tensor(seq) for seq in questions_indices]
answers_tensors = [torch.tensor(seq) for seq in answers_indices]

questions_padded = pad_sequence(questions_tensors, batch_first=True, padding_value=0)
answers_padded = pad_sequence(answers_tensors, batch_first=True, padding_value=0)

# 데이터셋 및 데이터로더 생성
train_data = TensorDataset(questions_padded, answers_padded)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [15]:
# Seq2Seq 모델 정의
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        x = self.embedding(x)
        outputs, (hn, cn) = self.lstm(x)
        return hn, cn

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden, cell):
        x = self.embedding(x)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        output = self.fc(output)
        return output, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        encoder_hidden, encoder_cell = self.encoder(src)
        decoder_output, _, _ = self.decoder(trg, encoder_hidden, encoder_cell)
        return decoder_output

In [16]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [17]:
embedding_dim = 64
hidden_dim = 128

encoder = Encoder(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim).to(device)
decoder = Decoder(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim).to(device)
model = Seq2Seq(encoder, decoder).to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss(ignore_index=0)  # 패딩 토큰 무시
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [18]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for src_batch, trg_batch in train_loader:
        src_batch, trg_batch = src_batch.to(device), trg_batch.to(device)
        optimizer.zero_grad()

        outputs = model(src_batch, trg_batch[:, :-1])
        loss = criterion(outputs.view(-1, vocab_size), trg_batch[:, 1:].contiguous().view(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}')

# 모델 저장
torch.save(model.state_dict(), 'seq2seq_model.pth')

Epoch 1/10, Loss: 4.715848580605722
Epoch 2/10, Loss: 3.2686726716212453
Epoch 3/10, Loss: 2.657492961497665
Epoch 4/10, Loss: 2.3744759352910036
Epoch 5/10, Loss: 2.209777685086851
Epoch 6/10, Loss: 2.0867512835932605
Epoch 7/10, Loss: 1.9857087967368219
Epoch 8/10, Loss: 1.9042404716069987
Epoch 9/10, Loss: 1.837182461181817
Epoch 10/10, Loss: 1.7788173804393392


In [19]:
# 에포크 횟수를 추가하기 위한 코드

encoder = Encoder(vocab_size, embedding_dim, hidden_dim)
decoder = Decoder(vocab_size, embedding_dim, hidden_dim)
model = Seq2Seq(encoder, decoder)

# 저장된 모델 불러오기
model.load_state_dict(torch.load('seq2seq_model.pth'))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

num_epochs = 40  # 추가 에포크 횟수

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for src_batch, trg_batch in train_loader:
        src_batch, trg_batch = src_batch.to(device), trg_batch.to(device)
        optimizer.zero_grad()

        outputs = model(src_batch, trg_batch[:, :-1])
        loss = criterion(outputs.view(-1, vocab_size), trg_batch[:, 1:].contiguous().view(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Additional Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}')

# 새로운 모델 다시 저장
torch.save(model.state_dict(), 'seq2seq_model_additional.pth')

Additional Epoch 1/40, Loss: 0.814550020489734
Additional Epoch 2/40, Loss: 0.41160575313375175
Additional Epoch 3/40, Loss: 0.3695628158532815
Additional Epoch 4/40, Loss: 0.3330538766404797
Additional Epoch 5/40, Loss: 0.2984240284631018
Additional Epoch 6/40, Loss: 0.26986531635498723
Additional Epoch 7/40, Loss: 0.24693450956158555
Additional Epoch 8/40, Loss: 0.2277318861744652
Additional Epoch 9/40, Loss: 0.21184836489530656
Additional Epoch 10/40, Loss: 0.19884725861423622
Additional Epoch 11/40, Loss: 0.18833810186842617
Additional Epoch 12/40, Loss: 0.17975857178815183
Additional Epoch 13/40, Loss: 0.17244873282020493
Additional Epoch 14/40, Loss: 0.16628499071765152
Additional Epoch 15/40, Loss: 0.16080207763121307
Additional Epoch 16/40, Loss: 0.15597886083959844
Additional Epoch 17/40, Loss: 0.15153529900906748
Additional Epoch 18/40, Loss: 0.14755604823114554
Additional Epoch 19/40, Loss: 0.14375404362173783
Additional Epoch 20/40, Loss: 0.14060581932005856
Additional Epoc

In [31]:
def predict(encoder, decoder, src_sentence, max_length=60):
    encoder.eval()
    decoder.eval()

    src_indices = text_to_indices(tokenize(src_sentence), word2idx)
    src_tensor = torch.tensor(src_indices).unsqueeze(0).to(device)

    with torch.no_grad():
        encoder_hidden, encoder_cell = encoder(src_tensor)

    trg_indices = [word2idx.get('<SOS>', 0)]
    trg_tensor = torch.tensor(trg_indices).unsqueeze(0).to(device)

    predicted_sentence = []

    for _ in range(max_length):
        with torch.no_grad():
            output, encoder_hidden, encoder_cell = decoder(trg_tensor, encoder_hidden, encoder_cell)
        next_token = output.argmax(2).item()

        # Ignore <UNK> tokens in the output
        if next_token != word2idx.get('<UNK>', 0):
            predicted_sentence.append(idx2word.get(next_token, '<UNK>'))

        trg_tensor = torch.tensor([[next_token]]).to(device)

        if next_token == word2idx.get('<EOS>', 0):
            break

    return ''.join(predicted_sentence)

In [35]:
sample_indices = rd.sample(range(len(df)), 5)

for idx in sample_indices:
    question = df['questions'].iloc[idx]
    real_answer = df['answers'].iloc[idx]
    predicted_answer = predict(encoder, decoder, question)

    print(f"질문: {question}")
    print(f"실제 답변: {real_answer}")
    print(f"예측 답변: {predicted_answer}")
    print('-' * 50)

질문: 연희 에스프레소 바의 주요 메뉴는 무엇이 있나요?
실제 답변: 연희 에스프레소 바의 메뉴에는 연희 에스프레소,  스트라파짜토이 있습니다
예측 답변:  가락 잠실 플래그점의 메뉴에는 딸기 그린거,  하나베이 있습니다
--------------------------------------------------
질문: 카츠젠 잠실나루점의 연락처가 어떻게 되나요?
실제 답변: 카츠젠 잠실나루점의 연락처는 02-415-9929입니다
예측 답변: 커피 신림점의 연락처는 0507-1315-8292입니다
--------------------------------------------------
질문: 생어거스틴 김포공항점의 주소는 어떻게 되나요?
실제 답변: 생어거스틴 김포공항점의 주소는 서울 강서구 하늘길 38입니다
예측 답변: 촌서명림 불광역점의 주소는 서울 금천구 시흥대로 253 1층 오스시스 1층입니다
--------------------------------------------------
질문: 노룬산분식의 주차시설이 있나요?
실제 답변: 노룬산분식의 주차시설이 있습니다
예측 답변: 로우켓는의 주차시설이 있습니다
--------------------------------------------------
질문: 짚신매운갈비찜 시흥사거리점의 영업시간은 몇시부터 몇시까지로 기재되어 있나요?
실제 답변: 짚신매운갈비찜 시흥사거리점의 영업시간은 11:30 - 24:00입니다
예측 답변: 신세기 영등포구청시청점의 영업시간은 11:00 - 22:00입니다
--------------------------------------------------
